# Évolution des Émissions des Transports Routiers : Analyse et Visualisation

L'étude des facteurs d'émissions des différents modes de transport routier est cruciale pour comprendre l'impact environnemental des transports. Les données utilisées proviennent de l'ADEME (Agence de l'Environnement et de la Maîtrise de l'Énergie) et sont disponibles à l'adresse https://data.ademe.fr/data-fair/api/v1/datasets/etude-facteurs-d'emissions-des-differents-modes-de-transport-routier/data-files/ADEME%20-%20FE_Transports%20routiers_v2.csv. Ce dataset contient des informations détaillées sur les émissions de polluants atmosphériques de différents types de véhicules routiers, organisées par année, type de véhicule, type de carburant, conditions de roulage et niveaux d'émissions de différents polluants.

L'objectif de cette analyse est de calculer la moyenne des émissions de CO, HC, NOx et particules par type de véhicule et par année, en distinguant les types de roulage. Cela nous permettra de comprendre l'évolution des émissions des transports routiers et d'identifier les domaines où des améliorations sont nécessaires.


## Méthodologie

La méthodologie utilisée consiste à importer les données dans un notebook Jupyter, puis à les analyser à l'aide de requêtes SQL avec DuckDB. Les données sont ensuite visualisées sous forme de graphiques interactifs avec Plotly Express. La requête SQL utilisée permet de calculer la moyenne des émissions de polluants par type de véhicule et par année, en distinguant les types de roulage. Les résultats sont ensuite triés par année, type de véhicule et type de roulage.


## Analyse et Visualisation

Les résultats de l'analyse sont présentés sous forme de graphiques qui montrent l'évolution des émissions moyennes par type de véhicule et contexte de roulage.


## Résultats

Les graphiques montrent que les émissions de polluants varient considérablement en fonction du type de véhicule, de l'année et des conditions de roulage. Les résultats détaillés sont présentés dans les graphiques interactifs.


## Conclusion

Cette analyse a permis de comprendre l'évolution des émissions des transports routiers et d'identifier les domaines où des améliorations sont nécessaires. Les résultats de cette étude peuvent être utilisés pour éclairer les décisions politiques et les choix technologiques visant à réduire l'impact environnemental des transports.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.ademe.fr/data-fair/api/v1/datasets/etude-facteurs-d'emissions-des-differents-modes-de-transport-routier/data-files/ADEME%20-%20FE_Transports%20routiers_v2.csv", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" SELECT 
  "Année", 
  "Véhicule", 
  "Roulage", 
  AVG(CAST("CO (g/km)" AS DOUBLE)) AS avg_CO,
  AVG(CAST("HC (g/km)" AS DOUBLE)) AS avg_HC,
  AVG(CAST("NOx (g/km)" AS DOUBLE)) AS avg_NOx,
  AVG(CAST("Particules (g/km)" AS DOUBLE)) AS avg_particules
FROM 
  loaded_dataset
GROUP BY 
  "Année", 
  "Véhicule", 
  "Roulage"
ORDER BY 
  "Année", 
  "Véhicule", 
  "Roulage" """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 176 lignes


,Année,Véhicule,Roulage,avg_CO,avg_HC,avg_NOx,avg_particules
0,2020,2RM,France,2.696,0.429,0.111,0.030
1,2020,2RM,Rural,3.438,0.400,0.123,0.029
2,2020,2RM,Urbain,1.572,0.458,0.090,0.031
3,2020,Autocar,France,1.315,0.092,3.502,0.055
4,2020,Autocar,Rural,0.984,0.072,2.534,0.043


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui est une surcouche de Plotly, idéale pour créer des visualisations interactives complexes de manière simple et intuitive. Plotly Express est adaptée ici pour représenter l'évolution des émissions moyennes de polluants par type de véhicule et contexte de roulage, avec une grande interactivité et des options de personnalisation. Cela permet une lecture détaillée et comparative des données.

In [4]:
import pandas as pd
import duckdb as ddb
import plotly.express as px
import pandas as pd

fig = px.line(
    df.sort_values(['Véhicule', 'Année']),
    x='Année',
    y=['avg_CO', 'avg_HC', 'avg_NOx', 'avg_particules'],
    color='Véhicule',
    facet_col='Roulage',
    labels={
        'value': 'Émissions (g/km)',
        'variable': 'Polluant',
        'Année': '',
        'Véhicule': 'Type&nbsp;véhicule',
        'Roulage': ''
    },
    title="Évolution des émissions moyennes par type véhicule et contexte de roulage",
    line_shape='spline',
    markers=True,
    hover_data={'Année': True, 'Roulage': True, 'Véhicule': True}
)

fig.update_traces(patch={'hovertemplate': (
    '<b>%{fullData.name}</b><br>Année : %{x}<br>'
    'Émissions : <b>%{y:.3f}</b> g/km<br>'
    'Roulage : %{customdata[0]}<extra></extra>'
)})

fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))

fig.update_layout(
    template='plotly_white',
    font=dict(family='Poppins', size=11),
    height=450,
    margin=dict(l=40, r=20, t=60, b=60),
    legend=dict(orientation='h', yanchor='bottom', y=-.2, x=.5, xanchor='center')
)

for axis in fig.select_xaxes():
    axis.update(tickmode='linear', tick0=2020, dtick=1)

dataviz = fig
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250808_163609.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250808_163609.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250808_163609.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250808_163609.png


--> Image Plotly sauvegardée avec succès.
